In [9]:
import mujoco
import glfw
import numpy as np
import time
from PIL import Image

XML_FILE_PATH = "/Users/edoardozappia/Desktop/Tesi_Magistrale/irregular_shape_2D_online.xml"

def main():

    dt = 0.01
    side_length = 0.6  # Lunghezza del lato del quadrato
    half_side = side_length / 2  # Metà lato (limiti del quadrato)
    alpha_x = 1
    alpha_y = 1
    beta_x = 0
    beta_y = 0
    std_x = 0.0    #std for gaussian noise on direction change and velocity. If 0.0 velocity is opposite
    std_y = 0.0    #std for gaussian noise on direction change and velocity. If 0.0 velocity is opposite

    if not glfw.init():
        print("Impossibile inizializzare GLFW")
        return

    glfw.window_hint(glfw.VISIBLE, glfw.TRUE)
    window = glfw.create_window(800, 800, "MuJoCo Viewer", None, None)
    if not window:
        print("Errore durante la creazione della finestra GLFW")
        glfw.terminate()
        return

    glfw.make_context_current(window)

    model = mujoco.MjModel.from_xml_path(XML_FILE_PATH)
    data = mujoco.MjData(model)

    # Configura la telecamera
    camera = mujoco.MjvCamera()
    camera.type = mujoco.mjtCamera.mjCAMERA_FREE
    camera.lookat = np.array([0, 0, 0])
    camera.distance = 2.0
    camera.azimuth = 90
    camera.elevation = -90

    # Renderer per visualizzare la finestra
    scene = mujoco.MjvScene(model, maxgeom=1000)
    context = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_150)

    t = 0  # Tempo iniziale
    
    positions = []

    while not glfw.window_should_close(window):

        t += dt

        # Posizione attuale
        x = data.qpos[3]
        y = data.qpos[4]

        # Salva la posizione corrente
        positions.append([x, y])

        # Rileva collisioni con i lati del quadrato
        if x >= half_side or x <= -half_side:
            alpha_x *= np.random.normal(-1, std_x)  # Inverti velocità lungo x
            print(f"Rimbalzo su lato verticale: x={x:.3f}, v_x={data.qvel[0]:.3f}")

        if y >= half_side or y <= -half_side:
            alpha_y *= np.random.normal(-1, std_y)  # Inverti velocità lungo y
            print(f"Rimbalzo su lato orizzontale: y={y:.3f}, v_y={data.qvel[1]:.3f}")

        # Calcola forze basate sulle velocità
        force_x = alpha_x * 150 + beta_x * np.random.normal(1, 0.5)
        force_y = alpha_y * 150 + beta_y * np.random.normal(1, 0.5)

        # Applica forze
        data.qfrc_applied[3] = force_x
        data.qfrc_applied[4] = force_y

        # Avanza la simulazione
        mujoco.mj_step(model, data)

        # Debug
        print(f"Posizione: x={x:.3f}, y={y:.3f}")
        print(f"Velocità: x_vel={data.qvel[0]:.3f}, y_vel={data.qvel[1]:.3f}")
        print(f"Forze: force_x={force_x:.3f}, force_y={force_y:.3f}")

        # Renderizza la scena
        mujoco.mjv_updateScene(model, data, mujoco.MjvOption(), None, camera, mujoco.mjtCatBit.mjCAT_ALL, scene)
        viewport_width, viewport_height = glfw.get_framebuffer_size(window)
        viewport = mujoco.MjrRect(0, 0, viewport_width, viewport_height)
        mujoco.mjr_render(viewport, scene, context)


        glfw.swap_buffers(window)
        glfw.poll_events()
        time.sleep(dt)
    
    # Salva il dataset delle posizioni target
    positions = np.array(positions)  # Converti in array NumPy
    print(f"Steps: {len(positions)}")
    #np.save(SAVE_PATH, positions)
    #print(f"Dataset delle posizioni target salvato in: {SAVE_PATH}")

    glfw.destroy_window(window)
    glfw.terminate()

if __name__ == "__main__":
    main()


Posizione: x=0.000, y=0.000
Velocità: x_vel=0.000, y_vel=0.000
Forze: force_x=150.000, force_y=150.000
Posizione: x=0.001, y=0.001
Velocità: x_vel=0.000, y_vel=0.000
Forze: force_x=150.000, force_y=150.000
Posizione: x=0.001, y=0.001
Velocità: x_vel=0.000, y_vel=0.000
Forze: force_x=150.000, force_y=150.000
Posizione: x=0.002, y=0.002
Velocità: x_vel=0.000, y_vel=0.000
Forze: force_x=150.000, force_y=150.000
Posizione: x=0.002, y=0.002
Velocità: x_vel=0.000, y_vel=0.000
Forze: force_x=150.000, force_y=150.000
Posizione: x=0.003, y=0.003
Velocità: x_vel=0.000, y_vel=0.000
Forze: force_x=150.000, force_y=150.000
Posizione: x=0.003, y=0.003
Velocità: x_vel=0.000, y_vel=0.000
Forze: force_x=150.000, force_y=150.000
Posizione: x=0.004, y=0.004
Velocità: x_vel=0.000, y_vel=0.000
Forze: force_x=150.000, force_y=150.000
Posizione: x=0.004, y=0.004
Velocità: x_vel=0.000, y_vel=0.000
Forze: force_x=150.000, force_y=150.000
Posizione: x=0.005, y=0.005
Velocità: x_vel=0.000, y_vel=0.000
Forze: for

: 